In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ============================
# CONFIGURACIÓN DEL LABERINTO
# ============================

def crear_laberinto(tamaño):
    """Crea un laberinto de tamaño NxN con obstáculos y una meta."""
    laberinto = np.zeros((tamaño, tamaño))
    # Generar obstáculos aleatorios
    for _ in range(int(tamaño * tamaño * 0.3)):  # 30% del laberinto son obstáculos
        x, y = np.random.randint(0, tamaño, size=2)
        laberinto[x, y] = 1
    # Asegurar que el inicio y la meta estén libres
    laberinto[0, 0] = 0  # Inicio
    laberinto[tamaño-1, tamaño-1] = 2  # Meta
    return laberinto

laberinto = crear_laberinto(100)

# Mostrar el laberinto inicial
plt.imshow(laberinto, cmap="gray")
plt.title("Laberinto inicial")
plt.show()

# ============================
# PARÁMETROS Y FUNCIONES BÁSICAS
# ============================

# Definimos las acciones: arriba, abajo, izquierda, derecha
acciones = ["arriba", "abajo", "izquierda", "derecha"]

def es_movimiento_valido(laberinto, pos_actual, accion):
    """Valida si un movimiento es válido dado el laberinto."""
    x, y = pos_actual
    if accion == "arriba" and x > 0 and laberinto[x-1, y] != 1:
        return (x-1, y)
    if accion == "abajo" and x < laberinto.shape[0]-1 and laberinto[x+1, y] != 1:
        return (x+1, y)
    if accion == "izquierda" and y > 0 and laberinto[x, y-1] != 1:
        return (x, y-1)
    if accion == "derecha" and y < laberinto.shape[1]-1 and laberinto[x, y+1] != 1:
        return (x, y+1)
    return pos_actual  # Si no es válido, mantener la posición



# ============================
# IMPLEMENTACIÓN DEL Q-LEARNING
# ============================

# Parámetros del modelo
alpha = 0.1  # Tasa de aprendizaje
gamma = 0.9  # Factor de descuento
epsilon = 0.1  # Exploración-explotación
episodios = 1000

# Inicializamos la tabla Q
q_table = np.zeros((*laberinto.shape, len(acciones)))

# Entrenamiento del agente
for episodio in range(episodios):
    pos_actual = (0, 0)  # El agente siempre comienza en la posición inicial
    while laberinto[pos_actual] != 2:  # Mientras no llegue a la meta
        # Elegir acción: explorar o explotar
        if np.random.random() < epsilon:  # Exploración
            accion = np.random.choice(range(len(acciones)))
        else:  # Explotación
            accion = np.argmax(q_table[pos_actual])

        # Determinar el siguiente estado
        nueva_pos = es_movimiento_valido(laberinto, pos_actual, acciones[accion])
        recompensa = 1 if laberinto[nueva_pos] == 2 else -0.01

        # Actualizar la tabla Q
        q_table[pos_actual + (accion,)] += alpha * (
            recompensa + gamma * np.max(q_table[nueva_pos]) - q_table[pos_actual + (accion,)]
        )

        # Mover al siguiente estado
        pos_actual = nueva_pos

# ============================
# EVALUACIÓN DEL MODELO
# ============================

def probar_modelo(laberinto, q_table):
    """Prueba el agente en el laberinto y devuelve el recorrido."""
    pos_actual = (0, 0)
    recorrido = [pos_actual]
    while laberinto[pos_actual] != 2:  # Mientras no llegue a la meta
        accion = np.argmax(q_table[pos_actual])  # Elegir la mejor acción
        pos_actual = es_movimiento_valido(laberinto, pos_actual, acciones[accion])
        recorrido.append(pos_actual)
    return recorrido

# Generar el recorrido del agente
recorrido = probar_modelo(laberinto, q_table)

def mostrar_recorrido(laberinto, recorrido):
    """Muestra el recorrido del agente en el laberinto."""
    laberinto_con_camino = laberinto.copy()
    for x, y in recorrido:
        laberinto_con_camino[x, y] = 0.5  # Marcar el camino
    plt.imshow(laberinto_con_camino, cmap="gray")
    plt.title("Recorrido del agente")
    plt.show()

# Mostrar el recorrido
mostrar_recorrido(laberinto, recorrido)


: 